In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import json

In [133]:
# Initiate headless driver for deployment
browser = Browser("chrome", executable_path="chromedriver", headless=True)


def load_browser(browser, url):
    browser.visit(url)
    
    return browser


def actions_used_by_recruiter():
    array_dic = []
    ### Total Usage:
    api_urls = [
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/actions/20180225/20190124',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/searches/20180225/20190124',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/emails/20180224/20190123',  
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/intakes/20180224/20190123',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/campaigns-sent/20180224/20190123',
    ]

    for api_url in api_urls:
        load_browser(browser, api_url)
        array_dic.append(get_data_as_json(browser))
    
    return array_dic
        
        
def most_viewed_resumes():

    ### Job Positions most viewed:
    api_url = 'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/resumes-stats/20180224/20190123'
    load_browser(browser, api_url)
    data = pd.DataFrame(get_data_as_json(browser))
    
    return data

def get_candidate_growth():
    
    ### Candidate Growth:
    api_url = 'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/candidates-count/20160101/20190123'
    load_browser(browser, api_url)
    data = pd.DataFrame(get_data_as_json(browser))
    data["mycand_member_growth"] = pd.to_numeric(data["mycand_member_growth"])
    data['yearmonth'] = pd.to_datetime(data['yearmonth'], format="%Y%m")
    data["month"] = data["yearmonth"].dt.month_name()
    data["year"] = data["yearmonth"].dt.year
    
    
    return data

def get_roi_data(browser):
    api_data = []
    
    api_urls = [
#         'https://recruitment-staging.ace.careerbuilder.com/api/cba/search/rdb/users/20180224/20190123',

### Total Usage:        
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/actions/20180225/20190124',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/searches/20180225/20190124',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/emails/20180224/20190123',  
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/intakes/20180224/20190123',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/campaigns-sent/20180224/20190123',
        
### Talent Network Membership:
#         'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/members/20190123',
#         'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/new-members/20180224/20190123',
        
### Candidate Growth:
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/candidates-count/20160101/20190123',

### Job Positions most viewed:
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/resumes-stats/20180224/20190123' #,

#         'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/candidates-stats/20180224/20190123',
#         'https://recruitment-staging.ace.careerbuilder.com/api/cba/roi/actions-comparison/20180224/20190123'    
    ]
    
    for api_url in api_urls:
        load_browser(browser, api_url)
        dict_data = get_data_as_json(browser)
#         df_data = pd.DataFrame(dict_data)
        api_data.append(dict_data)
#         api_data.append(pd.DataFrame.from_dict(dict_data)
    
    return api_data

def get_search_data(browser):
    api_data = []
    
    api_urls = [
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/search/tdp/users/20180225/20190124',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/search/tdp/license-usage/20180225/20190124',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/search/tdp/usage/20180225/20190124'
    ]
    
    for api_url in api_urls:
        load_browser(browser, api_url)
        api_data.append(get_data_as_json(browser))
    
    return api_data

def get_jobs_data(browser):
    api_data = []
    
    api_urls = [
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/jobs/category/20180225/20190124',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/jobs/location/20180225/20190124',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/jobs/header/20180225/20190124?categories=&locations=',
        'https://recruitment-staging.ace.careerbuilder.com/api/cba/jobs/monthly/20180225/20190124?categories=&locations='
    ]
    
    for api_url in api_urls:
        load_browser(browser, api_url)
        api_data.append(get_data_as_json(browser))
    
    return api_data



def login():
    login_url = "https://wwwtestaccounts.careerbuilder.com/share/login.aspx?next=https%3a%2f%2fwwwtestaccounts.careerbuilder.com%2fshare%2foauth2%2fauth.aspx%3fclient_id%3dCd03edb6f%26elui%3d1%26redirect_uri%3dhttps%25253a%25252f%25252frecruitment-staging.ace.careerbuilder.com%25252fauthCallback%25253foriginalUrl%25253d%2525252Fuserinfo%26response_type%3dcode%26scope%3dopenid&elui=1&client_id=Cd03edb6f"
    load_browser(browser, login_url)
        
    browser.fill_form({"cbsys_login_email": "Personified.Tester3@careerbuilder.com", "cbsys_login_password": "c0lumbusrocks!"}, form_id=None, name=None)
    browser.find_by_id('btnsigninemp').click()
    browser.is_text_present("username", wait_time=0.5)

    return True



def get_data_as_json(browser):

    html = browser.html
    api_result = BeautifulSoup(html, "html.parser")

    try:
        json_string = api_result.select_one("body").text

    except AttributeError:
        return None

    return json.loads(json_string)


In [134]:
login()

True

In [187]:
action_description = {"sd_searches": "Supply & Demand Searches",
    "intakes_started": "Intakes Started",
    "actions_taken": "Actions Taken",
    "campaigns_sent": "Campaigns Sent",
    "emails_sent": "Talent Network Emails Sent"}


array_dict = actions_used_by_recruiter()

data = pd.DataFrame.from_dict(array_dict[0], orient='index')
data = data.append(pd.DataFrame.from_dict(array_dict[1], orient='index'))
data = data.append(pd.DataFrame.from_dict(array_dict[2], orient='index'))
data = data.append(pd.DataFrame.from_dict(array_dict[3], orient='index'))
data = data.append(pd.DataFrame.from_dict(array_dict[4], orient='index'))

data = data.dropna()

data = data.rename(columns={0:"values"})
data["values"] = pd.to_numeric(data["values"])

data.sort_values(by="values", ascending=False)

response_ideas = [
    "Total usage since January 2018 is...",
]

for index, row in data.reset_index().iterrows():
    response_ideas.append("{} {:,.2f}".format(action_description[row[0]], row[1]))

response_ideas

['Total usage since January 2018 is...',
 'Actions Taken 36,912.00',
 'Supply & Demand Searches 92,146.00',
 'Intakes Started 51,373.00',
 'Campaigns Sent 401.00']

In [157]:
data

[{'actions_taken': '36759'},
 {'sd_searches': '91938'},
 {'emails_sent': None},
 {'intakes_started': '51368'},
 {'campaigns_sent': '401'}]

In [120]:
data = most_viewed_resumes()
data["position"] = data["onet_name"].str.split(" - ", n = 1, expand = True)[0]
data["city"] = data["onet_name"].str.split(" - ", n = 1, expand = True)[1]
data = data.drop(columns=["onet_name"])

data

position_1 = data.iloc[0]["position"]
city_1 = data.iloc[0]["city"]
viewed_1 = data.iloc[0]["resume_viewed"]
inventory_1 = data.iloc[0]["resume_inventory"]

position_2 = data.iloc[1]["position"]
city_2 = data.iloc[1]["city"]
viewed_2 = data.iloc[1]["resume_viewed"]
inventory_2 = data.iloc[1]["resume_inventory"]

position_3 = data.iloc[2]["position"]
city_3 = data.iloc[2]["city"]
viewed_3 = data.iloc[2]["resume_viewed"]
inventory_3 = data.iloc[2]["resume_inventory"]

response_ideas = [
    "Top 3 most viewed resumes are...",
    "{}. {}, with {} views, from an inventory of {}...".format(position_1, city_1, viewed_1, inventory_1),
    "{}. {}, with {} views, from {}...".format(position_2, city_2, viewed_2, inventory_2),
    "{}. {}, with {} views, from {}...".format(position_3, city_3, viewed_3, inventory_3)
]

response_ideas
# "Web Developer. CHICAGO, with 51 views from an inventory of 6025..."
# "General and Operations Manager. Atlanta, with 50 views, from an inventory of 5811..."

['Top 3 most viewed resumes are...',
 'Customer Service Representatives. MINEAPLS-ST. PL, with 60 views, from an inventory of 2256...',
 'Web Developers. CHICAGO, with 51 views, from 6025...',
 'General and Operations Managers. ATLANTA, with 50 views, from 5811...']

In [ ]:
data.sort_values(by=resume_viewed, ascending=False, inplace=True)


In [ ]:
import numpy as np
data = get_candidate_growth()
data
# np.sum(data["mycand_member_growth"])
# np.sum(data.iloc[0:])
# print(data)
# print("")
# print(data.iloc[0]["month"])
# print(data.iloc[0]["year"])
# print("")
# print(data.iloc[-1])
# data["yearmonth"].dt.month_name()


### Since January 2018 the growth in candidates has been variable with 
### a high pitch in April 2018 37.63% and the lowest in January 2018
### currently representing a lower growth of 7893

In [ ]:
first_date = "{} {}".format(data.iloc[0]["month"], data.iloc[0]["year"])
current_date = "{} {}".format(data.iloc[-1]["month"], data.iloc[-1]["year"])
current_growth = data.iloc[-1]["mycand_member_growth"]

data.sort_values(by="mycand_member_growth", ascending=False, inplace=True)

total = np.sum(data["mycand_member_growth"])

high_pitch = "{} {}".format(data.iloc[0]["month"], data.iloc[0]["year"])
high_pitch_value = data.iloc[0]["mycand_member_growth"]
high_pitch_value_mean = (high_pitch_value / total) * 100

lower_pitch = "{} {}".format(data.iloc[-1]["month"], data.iloc[-1]["year"])
lower_pitch_value = data.iloc[-1]["mycand_member_growth"]
lower_pitch_value_mean = (lower_pitch_value / total) * 100

response_ideas = [
    "Since {} the growth in candidates has been variable with".format(first_date), 
    "a high pitch in {} with {:.2f}% and the lowest in {}".format(high_pitch, high_pitch_value_mean, lower_pitch),
    "currently representing a candidate growth of {}".format(current_growth)
]

response_ideas

In [ ]:
get_roi_data(browser)

In [ ]:
get_search_data(browser)

In [ ]:
get_jobs_data(browser)

In [ ]:
from voiceit2 import VoiceIt2


In [ ]:
apiKey = 'key_27296bf11f754939add04c2dc4006e15'
apiToken = 'tok_52f75f2532bc4ce7beaacb54340548a8'

my_voiceit = VoiceIt2(apiKey, apiToken)
